# 05 Correct External Stations Centroid Connector Errors with Project Cards

In [1]:
import os
import sys
import yaml
import pickle
import glob

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

## Remote I/O and Parameters

In [4]:
# root_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'travel-model-two-networks')
root_dir = "D:/github/lfs-cleaning/travel-model-two-networks"
input_dir = os.path.join(root_dir, 'data', 'processed', 'version_04')
output_dir = os.path.join(root_dir, 'data', 'processed', 'version_04')
card_dir = os.path.join(root_dir, 'project_cards')
# lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'mtc-Lasso')
lasso_dir = "Z:/Data/Users/Sijia/MTC/github/Lasso"

In [5]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2021-02-19 17:28:04, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-19 17:28:04, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-19 17:28:04, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-19 17:28:04, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso


## Read Version 04 Scenario

In [6]:
version_04_pickle_file_name = os.path.join(input_dir, 'working_scenario_04.pickle')
v_04_scenario = pickle.load(open(version_04_pickle_file_name, 'rb'))

In [7]:
v_04_scenario.road_net.links_df.shape

(1634769, 43)

In [8]:
v_04_scenario.transit_net.feed.routes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 701 entries, 0 to 700
Data columns (total 16 columns):
route_id_original         701 non-null object
agency_id                 604 non-null object
route_short_name          665 non-null object
route_long_name           679 non-null object
route_desc                14 non-null object
route_type                701 non-null int64
route_url                 217 non-null object
route_color               260 non-null object
route_text_color          243 non-null object
agency_raw_name           701 non-null object
route_sort_order          95 non-null object
min_headway_minutes       32 non-null object
eligibility_restricted    4 non-null object
continuous_pickup         4 non-null object
continuous_drop_off       4 non-null object
route_id                  701 non-null object
dtypes: int64(1), object(15)
memory usage: 87.8+ KB


In [9]:
v_04_scenario.road_net = mtc.add_centroid_and_centroid_connector(
    roadway_network = v_04_scenario.road_net,
    parameters = parameters)

2021-02-19 17:31:07, INFO: Adding centroid and centroid connector to standard network
2021-02-19 17:31:07, INFO: Adding centroid and centroid connector to standard network
2021-02-19 17:31:07, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-19 17:31:07, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-19 17:31:07, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-19 17:31:07, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-19 17:31:18, INFO: Finished adding centroid and centroid connectors
2021-02-19 17:31:18, INFO: Finished adding centroid and centroid connectors


In [10]:
v_04_scenario.road_net.nodes_df[
    v_04_scenario.road_net.nodes_df.model_node_id<100000
].info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 4789 entries, 644475 to 653383
Data columns (total 12 columns):
osm_node_id      0 non-null float64
shst_node_id     0 non-null object
county           0 non-null object
drive_access     4789 non-null int64
walk_access      4789 non-null int64
bike_access      4789 non-null int64
model_node_id    4789 non-null int64
rail_only        4789 non-null int64
X                4789 non-null float64
Y                4789 non-null float64
geometry         4789 non-null geometry
farezone         0 non-null float64
dtypes: float64(4), geometry(1), int64(5), object(2)
memory usage: 486.4+ KB


In [13]:
v_04_scenario.road_net.nodes_df[
    v_04_scenario.road_net.nodes_df.model_node_id.isin([900020, 2192724])
]

,osm_node_id,shst_node_id,county,drive_access,walk_access,bike_access,model_node_id,rail_only,X,Y,geometry,farezone
643638,303938467.0,9085180a41d6019b3a18be7d3811675c,Santa Clara,1,1,1,2192724,0,-122.120978,37.257668,POINT (-122.12098 37.25767),NaN
649229,NaN,NaN,NaN,1,1,1,900020,0,-122.122228,37.258329,POINT (-122.12223 37.25833),NaN


## Create Scenario and Apply Project Cards

In [11]:
v_04_scenario.road_net.links_df["id"] = v_04_scenario.road_net.links_df["shstGeometryId"]

In [12]:
v_05_scenario = Scenario.create_scenario(
    base_scenario= v_04_scenario,
    card_directory = card_dir,
    tags = ['External Stations Review'],
    validate_project_cards=False)

2021-02-19 17:33:10, INFO: Creating Scenario
2021-02-19 17:33:10, INFO: Creating Scenario


In [14]:
v_05_scenario.road_net.links_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2190409 entries, 0 to 2190408
Data columns (total 44 columns):
access                object
bike_access           object
drive_access          object
fromIntersectionId    object
lanes                 object
maxspeed              object
name                  object
oneWay                object
ref                   object
roadway               object
shstGeometryId        object
shstReferenceId       object
toIntersectionId      object
u                     float64
v                     float64
walk_access           object
wayId                 object
county                object
model_link_id         int64
A                     int64
B                     int64
rail_traveltime       float64
rail_only             int64
locationReferences    object
shape_id              float64
geometry              geometry
ft_cal                float64
ft                    int64
useclass              float64
assignable            float64
trans

In [15]:
v_05_scenario.apply_all_projects()

2021-02-19 17:34:18, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
2021-02-19 17:34:18, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
2021-02-19 17:34:18, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
2021-02-19 17:34:18, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
2021-02-19 17:34:18, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
2021-02-19 17:34:18, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB


c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


2021-02-19 17:36:38, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 NB
2021-02-19 17:36:38, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 NB
2021-02-19 17:36:38, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 NB
2021-02-19 17:36:38, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 NB
2021-02-19 17:36:38, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 NB
2021-02-19 17:36:38, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 NB
2021-02-19 17:38:55, INFO: Applying Year 2015 Add centroid connector at External Station connecting Vic Fazio Highway CA-1

2021-02-19 17:59:48, INFO: Applying Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 NB
2021-02-19 17:59:48, INFO: Applying Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 NB
2021-02-19 17:59:48, INFO: Applying Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 NB
2021-02-19 17:59:48, INFO: Applying Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 NB
2021-02-19 17:59:48, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 NB
2021-02-19 17:59:48, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 NB
2021-02-19 18:02:07, INFO: Applying Year 2015 Add centroid connector at External Station connecting Pacheco Pass Highway CA-156 SB
2021-02-19 18:02:07, INFO: Applying Year 2015 Add centroi

2021-02-19 18:22:42, INFO: Applying Year 2015 Add centroid connector at External Station connecting Dwight D Eisenhower Highway I-80 EB
2021-02-19 18:22:42, INFO: Applying Year 2015 Add centroid connector at External Station connecting Dwight D Eisenhower Highway I-80 EB
2021-02-19 18:22:42, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting Dwight D Eisenhower Highway I-80 EB
2021-02-19 18:22:42, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting Dwight D Eisenhower Highway I-80 EB
2021-02-19 18:24:55, INFO: Applying Year 2015 Add centroid connector at External Station connecting Cabrillo Highway CA-1 SB
2021-02-19 18:24:55, INFO: Applying Year 2015 Add centroid connector at External Station connecting Cabrillo Highway CA-1 SB
2021-02-19 18:24:55, INFO: Applying Year 2015 Add centroid connector at External Station connecting Cabrillo Highway CA-1 SB
2021-02-19 18:24:55, INFO

In [16]:
v_05_scenario.applied_projects

['Year 2015 Modify Facility Type to Service',
 'Year 2015 Modify Facility Type to Ramp shape incorrect',
 'Year 2015 Modify Facility Type to Ramp',
 'Year 2015 Modify Lanes to 2 Lane and Facility Type to Ramp shape incorrect',
 'Year 2015 Modify Lanes to 2 Lane and Facility Type to Ramp',
 'Year 2015 Modify Lanes to 1 Lane and Facility Type to Ramp',
 'Year 2015 Bus Only Segments',
 'Year 2015 Modify Lanes to 7 Lanes',
 'Year 2015 Modify Lanes to 6 Lanes',
 'Year 2015 Modify Lanes to 5 Lanes',
 'Year 2015 Modify Lanes to 4 Lanes',
 'Year 2015 Modify Lanes to 3 Lanes',
 'Year 2015 Modify Lanes to 2 Lanes',
 'Year 2015 Modify Lanes to 1 Lane',
 'Year 2015 Modify Lanes to 14 Lanes',
 'Year 2015 Modify HOV Lanes and USE on US-101 SB from South of Healdsburg to North of Petaluma',
 'Year 2015 Modify USE and HOV Lanes on US-101 SB (Segment 02) from South of Healdsburg to North of Petaluma',
 'Year 2015 Modify HOV Lanes and USE on US 101 SB CA-85 to Oregon Expressway',
 'Year 2015 Modify HOV 

In [17]:
v_05_scenario.road_net.links_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2190435 entries, 0 to 2190434
Data columns (total 45 columns):
access                object
bike_access           object
drive_access          object
fromIntersectionId    object
lanes                 object
maxspeed              object
name                  object
oneWay                object
ref                   object
roadway               object
shstGeometryId        object
shstReferenceId       object
toIntersectionId      object
u                     object
v                     object
walk_access           object
wayId                 object
county                object
model_link_id         int64
A                     int64
B                     int64
rail_traveltime       object
rail_only             object
locationReferences    object
shape_id              object
geometry              geometry
ft_cal                object
ft                    int64
useclass              object
assignable            object
transit    

In [18]:
v_05_scenario.road_net.links_df.tail().model_link_id

2190430    26067354
2190431    26067353
2190432    26067352
2190433    26067351
2190434    26067350
Name: model_link_id, dtype: int64

In [19]:
print(v_05_scenario.road_net.links_df[
    v_05_scenario.road_net.links_df.model_link_id == 26067343
].geometry.iloc[0])

LINESTRING (-122.1222278205091 37.25832914628474, -122.120978 37.2576679)


In [20]:
v_05_scenario.road_net.links_df[
    v_05_scenario.road_net.links_df.model_link_id == 26067343
].iloc[0][["A", "B"]]

A     900020
B    2192724
Name: 2190415, dtype: object

## Make Travel Model Networks

In [66]:
model_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = v_05_scenario.road_net, 
    parameters = parameters)

2021-02-22 10:45:56, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-22 10:45:56, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-22 10:45:56, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-22 10:45:56, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso


In [67]:
print(model_net.links_df.shape)
print(model_net.nodes_df.shape)
print(model_net.shapes_df.shape)

(2190435, 45)
(688957, 12)
(1147816, 7)


In [68]:
model_net.links_df.model_link_id.nunique()

2190435

In [69]:
model_net = mtc.add_tap_and_tap_connector(
    roadway_network = model_net,
    parameters = parameters)

2021-02-22 10:46:12, INFO: Adding tap and tap connector to standard network
2021-02-22 10:46:12, INFO: Adding tap and tap connector to standard network
2021-02-22 10:46:12, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-22 10:46:12, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-22 10:46:12, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-22 10:46:12, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-22 10:46:19, INFO: Finished adding tap and tap connectors
2021-02-22 10:46:19, INFO: Finished adding tap and tap connectors


In [70]:
print(model_net.links_df.shape)
print(model_net.nodes_df.shape)
print(model_net.shapes_df.shape)

(2225677, 45)
(694957, 12)
(1165437, 7)


In [71]:
model_net.links_df.model_link_id.nunique()

2225677

In [72]:
model_net.nodes_df.model_node_id.nunique()

694957

In [73]:
model_net.shapes_df.id.nunique()

1165437

In [74]:
model_net = mtc.roadway_standard_to_mtc_network(model_net, parameters)

2021-02-22 10:46:41, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2021-02-22 10:46:41, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2021-02-22 10:46:41, INFO: Creating managed lane network.
2021-02-22 10:46:41, INFO: Creating managed lane network.
2021-02-22 10:46:41, INFO: Creating network with duplicated managed lanes
2021-02-22 10:46:41, INFO: Creating network with duplicated managed lanes


c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
c:\users\wang

2021-02-22 11:03:14, INFO: Determining cntype
2021-02-22 11:03:14, INFO: Determining cntype
2021-02-22 11:03:14, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-22 11:03:14, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-22 11:03:14, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-22 11:03:14, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-22 11:03:14, INFO: Variable 'cntype' already in network. Returning without overwriting.
2021-02-22 11:03:14, INFO: Variable 'cntype' already in network. Returning without overwriting.
2021-02-22 11:03:14, INFO: Determining transit
2021-02-22 11:03:14, INFO: Determining transit
2021-02-22 11:03:14, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-22 11:03:14, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-22 11:03:14, INFO: Lasso base directory set as: Z:/Data/

In [75]:
model_net.links_df.shape

(2230079, 57)

In [76]:
model_net.links_df.model_link_id.nunique()

2230079

In [77]:
model_net.nodes_mtc_df.shape

(696464, 13)

In [78]:
model_net.nodes_df.model_node_id.nunique()

696464

## Write to Disk

## As shapefile

In [81]:
model_net.write_roadway_as_shp(
    output_link_shp = os.path.join(output_dir, 'links.shp'),
    output_node_shp = os.path.join(output_dir, 'nodes.shp'),
    link_output_variables = ["model_link_id", "A", "B", "geometry", 
                             "drive_access",
                            'lanes_EA', 'lanes_AM',
       'lanes_MD', 'lanes_PM', 'lanes_EV'],
    node_output_variables = ["model_node_id", "geometry"],
    data_to_csv = False,
    data_to_dbf = True,
)

2021-02-22 12:57:49, INFO: Writing Network as Shapefile
2021-02-22 12:57:49, INFO: Writing Network as Shapefile
2021-02-22 12:57:49, INFO: Renaming DBF Node Variables
2021-02-22 12:57:49, INFO: Renaming DBF Node Variables
2021-02-22 12:57:49, INFO: Renaming variables so that they are DBF-safe
2021-02-22 12:57:49, INFO: Renaming variables so that they are DBF-safe
2021-02-22 12:58:01, INFO: Renaming DBF Link Variables
2021-02-22 12:58:01, INFO: Renaming DBF Link Variables
2021-02-22 12:58:01, INFO: Renaming variables so that they are DBF-safe
2021-02-22 12:58:01, INFO: Renaming variables so that they are DBF-safe
2021-02-22 12:58:05, INFO: Writing Node Shapes:
 - D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_04\nodes.shp
2021-02-22 12:58:05, INFO: Writing Node Shapes:
 - D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_04\nodes.shp
2021-02-22 12:58:49, INFO: Writing Link Shapes:
 - D:/github/lfs-cleaning/travel-model-two-networks\data\pro

## As fixed width (for CUBE)

In [93]:
model_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, 'links.txt'),
    output_node_txt = os.path.join(output_dir, 'nodes.txt'),
    output_link_header_width_txt = os.path.join(output_dir, 'links_header_width.txt'),
    output_node_header_width_txt = os.path.join(output_dir, 'nodes_header_width.txt'),
    output_cube_network_script = os.path.join(output_dir, 'make_complete_network_from_fixed_width_file.s'),
    #drive_only = True
)

2021-02-19 16:38:24, INFO: Starting fixed width conversion
2021-02-19 16:38:24, INFO: Starting fixed width conversion
2021-02-19 17:02:27, INFO: Writing out link database
2021-02-19 17:02:27, INFO: Writing out link database
2021-02-19 17:02:42, INFO: Writing out link header and width ----
2021-02-19 17:02:42, INFO: Writing out link header and width ----
2021-02-19 17:02:42, INFO: Starting fixed width conversion
2021-02-19 17:02:42, INFO: Starting fixed width conversion
2021-02-19 17:04:45, INFO: Writing out node database
2021-02-19 17:04:45, INFO: Writing out node database
2021-02-19 17:04:47, INFO: Writing out node header and width
2021-02-19 17:04:47, INFO: Writing out node header and width


In [94]:
%%capture
os.chdir(output_dir)
!make_cube.bat

LINKMERGE: 3056897-3009085     

LINKMERGE: 3057208-400452     

LINKMERGE: 3057508-3047210     

LINKMERGE: 3057817-3068468     

LINKMERGE: 3058134-3093796     

LINKMERGE: 3058446-3067096     

LINKMERGE: 3058768-3037044     

LINKMERGE: 3059092-3026786     


In [95]:
%%capture
!runtpp make_simple_roadway_network.s
!runtpp make_complete_marin_network.s
!runtpp make_simple_transit_network.s
!runtpp make_highway_ext_stn_connector_network.s


LINKMERGE: 3059416-3029385     

LINKMERGE: 3059737-3026682     

LINKMERGE: 3060056-3029491     

LINKMERGE: 3060363-3020956     

LINKMERGE: 3060668-3020320     

LINKMERGE: 3060986-3000907     

LINKMERGE: 3061308-423761     

LINKMERGE: 3061620-3032015     

LINKMERGE: 3061944-3019315     

LINKMERGE: 3062255-410765     

LINKMERGE: 3062578-3009452     

LINKMERGE: 3062892-3003343     

LINKMERGE: 3063197-3002393     

LINKMERGE: 3063526-3049875     

LINKMERGE: 3063862-3054726     

LINKMERGE: 3064181-3028159     

LINKMERGE: 3064507-3023128     

LINKMERGE: 3064829-3020521     

LINKMERGE: 3065160-3001181     

LINKMERGE: 3065476-3022014     

LINKMERGE: 3065803-3007802     

LINKMERGE: 3066132-3018456     

LINKMERGE: 3066459-3038733     

LINKMERGE: 3066787-423245     

LINKMERGE: 3067115-3064469     

LINKMERGE: 3067443-3069914     

LINKMERGE: 3067774-3064263     

LINKMERGE: 3068108-423870     

LINKMERGE: 3068422-3053293     

LINKMERGE: 3068741-424353     

LINKMERGE: 306

## As transit line file (for CUBE)

In [33]:
standard_transit_net = StandardTransit.fromTransitNetwork(v_05_scenario.transit_net, parameters = parameters)
mtc.write_as_cube_lin(standard_transit_net, parameters, outpath = os.path.join(output_dir, "transit.lin"))

2021-02-11 18:28:53, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-11 18:28:53, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-11 18:28:53, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-11 18:28:53, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-11 18:28:53, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-02-11 18:28:53, INFO: Converting GTFS Standard Properties to MTC's Cube Standard


## As pickle

In [89]:
working_scenario_filename = os.path.join(output_dir, 'working_scenario_05.pickle')
pickle.dump(v_05_scenario, open(working_scenario_filename, 'wb'))

## As standard network

In [35]:
model_net.write(path = output_dir, filename = "v04")

In [90]:
parameters.time_period_to_time

{'EA': ('3:00', '6:00'),
 'AM': ('6:00', '10:00'),
 'MD': ('10:00', '15:00'),
 'PM': ('15:00', '19:00'),
 'EV': ('19:00', '3:00')}

In [91]:
from network_wrangler.utils import parse_time_spans

In [97]:
parse_time_spans(("15:00", "19:00"))

(54000, 68400)

In [98]:
parse_time_spans((54000, 68400))

(54000, 68400)